In [21]:
import pandas as pd
import numpy as np
from PIL import Image
import io
import os

# Load Parquet file into a DataFrame
df = pd.read_parquet("../data/processed_sticker_dataset.parquet")

# df["combined_embedding"] = df["combined_embedding"].apply(lambda x: np.array(x, dtype=np.float32))

# # Assuming 'image' column contains base64 or binary data:
# # If it's base64-encoded
# # def decode_image_base64(base64_string):
# #     from io import BytesIO
# #     from base64 import b64decode
# #     image_data = b64decode(base64_string)
# #     image = Image.open(BytesIO(image_data))
# #     return image

# # Or if images are stored as binary blobs
# def decode_image_binary(binary_data):
#     image = Image.open(io.BytesIO(binary_data))
#     return image

# # # For base64 example
# # df['decoded_images'] = df['image_path'].apply(decode_image_base64)
# # For binary blobs
# df['decoded_images'] = df['combined_embedding'].apply(decode_image_binary)

# # Show the first image from your dataset
# df['decoded_images'][0].show()


In [16]:
print(df.head())

                                  combined_embedding  \
0  [0.05615041, 0.06784809, -0.03342954, 0.037553...   
1  [-0.124234326, 0.07463956, -0.011985385, 0.004...   
2  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
3  [-0.06495428, -0.04292713, 0.013164402, 0.0220...   
4  [0.027918205, 0.075559475, 0.03622711, -0.0181...   

                                          image_path  
0  ../data/tensor_images/AlexatorStickers\cartoon...  
1  ../data/tensor_images/AlexatorStickers\cartoon...  
2  ../data/tensor_images/AlexatorStickers\cartoon...  
3  ../data/tensor_images/AlexatorStickers\cartoon...  
4  ../data/tensor_images/AlexatorStickers\cartoon...  


In [24]:
# from diffusers import StableDiffusionPipeline
# import torch
# from PIL import Image
# import requests
# from io import BytesIO

# # Load the Stable Diffusion model (make sure you have a Hugging Face API key if needed)
# model = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v-1-4-original")
# model.to("cuda" if torch.cuda.is_available() else "cpu")

# # Example: Process each row in the DataFrame
# for index, row in df.iterrows():
#     # Extract embedding and image path (you can skip embedding if you're using the path directly)
#     image_path = row['image_path']
    
#     # If you want to generate from the embedding, use it as the input (e.g., for text-to-image generation)
#     # Alternatively, you can generate based on image path if you have it

#     # Load image (if you have a direct path to the image file)
#     if image_path.startswith("http"):
#         response = requests.get(image_path)
#         img = Image.open(BytesIO(response.content))
#     else:
#         img = Image.open(image_path)

#     # Generate a sticker from the image using LDM or an additional text prompt
#     prompt = "A cute sticker with the image"
#     generated_img = model(prompt).images[0]

#     # Save or display the generated sticker
#     generated_img.save(f"sticker_{index}.png")
#     generated_img.show()


In [ ]:
import torch
from diffusers import DiffusionPipeline

# Load the smallest LDM model
pipe = DiffusionPipeline.from_pretrained("CompVis/ldm-text2im-large-256", torch_dtype=torch.float32)

# Move model to CPU
pipe.to("cpu")

print("Model running on:", pipe.device)

for index, row in df.iterrows():
    # Extract the image path from the Parquet file (local file path)
    image_path = row['image_path']

    # 4. Check if the image path is a tensor file (.pt)
    if image_path.endswith(".pt"):
        try:
            # Load the tensor from the .pt file
            tensor_image = torch.load(image_path)

            # Ensure the tensor is in the correct shape (e.g., C x H x W)
            if tensor_image.ndimension() == 3:  # Check if it's a 3D tensor (C, H, W)
                # Convert the tensor to a PIL image (assuming the tensor is in the range [0, 1])
                tensor_image = tensor_image.permute(1, 2, 0)  # Convert from C x H x W to H x W x C
                tensor_image = tensor_image.numpy()  # Convert to numpy array
                pil_image = Image.fromarray((tensor_image * 255).astype('uint8'))  # Convert to uint8 for PIL
            else:
                print(f"Warning: Tensor shape of image '{image_path}' is not valid. Skipping.")
                continue

        except Exception as e:
            print(f"Error loading tensor image from {image_path}: {e}")
            continue

    else:
        # If the image path is not a tensor file, assume it's a normal image file
        if os.path.exists(image_path):
            pil_image = Image.open(image_path)
        else:
            print(f"Warning: Image path '{image_path}' does not exist. Skipping.")
            continue

prompt = "unicorn wearing suit"
image = pipe(prompt, guidance_scale=7.5).images[0]

# Save & Show Image
image.save("cpu_generated_sticker.png")
image.show()



Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

An error occurred while trying to fetch C:\Users\97798\.cache\huggingface\hub\models--CompVis--ldm-text2im-large-256\snapshots\30de525ca11a880baea4962827fb6cb0bb268955\unet: Error no file named diffusion_pytorch_model.safetensors found in directory C:\Users\97798\.cache\huggingface\hub\models--CompVis--ldm-text2im-large-256\snapshots\30de525ca11a880baea4962827fb6cb0bb268955\unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch C:\Users\97798\.cache\huggingface\hub\models--CompVis--ldm-text2im-large-256\snapshots\30de525ca11a880baea4962827fb6cb0bb268955\vqvae: Error no file named diffusion_pytorch_model.safetensors found in directory C:\Users\97798\.cache\huggingface\hub\models--CompVis--ldm-text2im-large-256\snapshots\30de525ca11a880baea4962827fb6cb0bb268955\vqvae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
The config attributes {'timestep_values': None} were p

Model running on: cpu


  0%|          | 0/50 [00:00<?, ?it/s]